In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
def mall_wifi_dateframe(mall_id, path):
    cols = pickle.load(open('../cjy/input/'+path+'/'+mall_id+'_cols.p','rb'))
    coo = pickle.load(open('../cjy/input/'+path+'/'+mall_id+'.p','rb'))
    coo = coo.tocoo(copy=False)
    coo_group = pd.DataFrame({'row': coo.row, 'col': coo.col, 'data': coo.data}
                 )[['row', 'col', 'data']].sort_values(['row', 'col']
                 ).reset_index(drop=True).groupby('col')
    df = pd.DataFrame(0 * np.ones((max(coo.row)+1, len(cols))), columns=cols, dtype=np.int64)
    tmp = coo_group.apply(lambda x:x.row.values)
    for col,rows,values in zip(tmp.index.tolist(),tmp.values,
                                coo_group.apply(lambda x:x.data.tolist())):
        df.iloc[rows,col]=values
    return df

In [3]:
train = pd.read_csv('../input/训练数据-ccf_first_round_user_shop_behavior.csv')
test = pd.read_csv('../input/AB榜测试集-evaluation_public.csv')

train['row_id'] = -1
test['shop_id'] = ''
train = train[['row_id', 'shop_id']]
test = test[['row_id', 'shop_id']]

train.shape
train.head()
test.shape
test.head()

(1138015, 2)

,row_id,shop_id
0,-1,s_2871718
1,-1,s_2871718
2,-1,s_181637
3,-1,s_609470
4,-1,s_3816766


(483931, 2)

,row_id,shop_id
0,118742,
1,118743,
2,118744,
3,118745,
4,118746,


In [4]:
full_data = pd.concat([train, test])
full_data.reset_index(drop=True, inplace=True)

full_data.shape
full_data.head()
full_data.tail()

del train
del test

(1621946, 2)

,row_id,shop_id
0,-1,s_2871718
1,-1,s_2871718
2,-1,s_181637
3,-1,s_609470
4,-1,s_3816766


,row_id,shop_id
1621941,483927,
1621942,483928,
1621943,483929,
1621944,483930,
1621945,483931,


In [8]:
shop = pd.read_csv('../input/训练数据-ccf_first_round_shop_info.csv')

mall_ids = set(shop.mall_id)
del shop

In [6]:
import os  
  
finish_mall = os.listdir('./model/xgb10/')
# finish_mall.remove('.ipynb_checkpoints')
# finish_mall.remove('submit')
# finish_mall.remove('result')
for i in range(len(finish_mall)):
    finish_mall[i] = finish_mall[i][:-2]
finish_mall

['m_2123',
 'm_1790',
 'm_6803',
 'm_8093',
 'm_2267',
 'm_5085',
 'm_4828',
 'm_5767',
 'm_1377',
 'm_2907',
 'm_5810',
 'm_909',
 'm_623',
 'm_4548',
 'm_3832',
 'm_3528',
 'm_2467',
 'm_7523',
 'm_4422',
 'm_5154',
 'm_625',
 'm_1021',
 'm_979',
 'm_5529',
 'm_2715',
 'm_4121',
 'm_7994',
 'm_2333',
 'm_4168',
 'm_4406',
 'm_3501',
 'm_3054',
 'm_1089',
 'm_4515',
 'm_4759',
 'm_626',
 'm_7973',
 'm_4572',
 'm_2009',
 'm_6167',
 'm_968',
 'm_3739',
 'm_4033',
 'm_4094',
 'm_1409',
 'm_3445',
 'm_1293',
 'm_2578',
 'm_3916',
 'm_5352',
 'm_8344',
 'm_3517',
 'm_9068',
 'm_3871',
 'm_615',
 'm_1375',
 'm_1920',
 'm_3019',
 'm_6587',
 'm_3313',
 'm_1621',
 'm_4341',
 'm_4187',
 'm_4011',
 'm_7168',
 'm_1950',
 'm_5825',
 'm_2058',
 'm_822',
 'm_622',
 'm_2415',
 'm_2182',
 'm_4459',
 'm_690',
 'm_1085',
 'm_1263',
 'm_7601',
 'm_5892',
 'm_5076',
 'm_4543',
 'm_4495',
 'm_2270',
 'm_9054',
 'm_4079',
 'm_2224',
 'm_1831',
 'm_2878',
 'm_1175',
 'm_3112',
 'm_6337',
 'm_3425',
 'm_8379'

In [7]:
len(finish_mall)

97

In [9]:
for mall_id in tqdm(mall_ids):
    if mall_id in finish_mall:
        y_pred_proba = pickle.load(open('./model/xgb10/' + str(mall_id) + '.p', 'rb'))
        x_test = pickle.load(open('./output/test_user_neg10/' + str(mall_id) + '.p', 'rb'))

        y_max = []
        for i in range(y_pred_proba.shape[0]):
            if i % 9 == 0:
                y_max.append(i)
            else:
                y_max.append(0)

        y_pred = {'test_full_order' : [],
                 'train_full_order' : []}
        for i in range(len(y_max)):
            if i%9 == 0:
                y_pred['test_full_order'].append(x_test.iloc[0, int(i/9)].copy())
                y_pred['train_full_order'].append(x_test.iloc[y_max[i] % 9 + 1, int(i/9)].copy())
        
        pickle.dump(pd.DataFrame(y_pred), open('./model/temp/' + str(mall_id) + '.p', 'wb'), protocol=4)

100%|██████████| 97/97 [15:58<00:00,  9.88s/it]


In [10]:
result = {'row_id' : [],
          'shop_id' : []}

for mall_id in tqdm(mall_ids):
    if mall_id in finish_mall:
        m = pickle.load(open('./model/temp/' + mall_id + '.p', 'rb'))
        m = m.as_matrix()
        for i in range(len(m)):
            result['row_id'].append(full_data.iloc[m[i, 0]].row_id)
            result['shop_id'].append(full_data.iloc[m[i, 1]].shop_id)
        
# pickle.dump(pd.DataFrame(result), open('./model/result/submit_xgb_onehot.csv', 'wb'), protocol=4)

100%|██████████| 97/97 [10:32<00:00,  6.53s/it]


In [11]:
result = pd.DataFrame(result)
result.index = result.row_id.tolist()
result.columns = ['row_id', 'shop_id']

result.shape
len(set(result.row_id))
result.head()

(483931, 2)

483931

,row_id,shop_id
118833,118833,s_512700
118851,118851,s_506597
118916,118916,s_3726779
118989,118989,s_525779
118990,118990,s_2939231


In [12]:
result.to_csv('./model/submit/CosNN.csv', index=False)